### Q1. Downloading the data
We'll use the same NYC taxi dataset(https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page), but instead of "Green Taxi Trip Records", we'll use "For-Hire Vehicle Trip Records".

Download the data for January and February 2021.

Note that you need "For-Hire Vehicle Trip Records", not "High Volume For-Hire Vehicle Trip Records".

Read the data for January. How many records are there?

In [1]:
!python -V

Python 3.9.12


In [135]:
import pandas as pd


In [136]:
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

#### Q1. Download the data for January and February 2021.

Note that you need "For-Hire Vehicle Trip Records", not "High Volume For-Hire Vehicle Trip Records".

Read the data for January. How many records are there?

1054112 <br>
1154112 <br>
1254112 <br>
1354112 <br>

Ans: 1154112

In [137]:
df = pd.read_parquet('./data/fhv_tripdata_2021-01.parquet')

In [138]:
df.count()

dispatching_base_num      1154112
pickup_datetime           1154112
dropOff_datetime          1154112
PUlocationID               195845
DOlocationID               991892
SR_Flag                         0
Affiliated_base_number    1153227
dtype: int64

In [139]:
df.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009
2,B00013,2021-01-01 00:01:00,2021-01-01 01:51:00,NaN,NaN,None,B00013
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,None,B00037
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,None,B00037


#### Q2. Computing duration
Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the average trip duration in January?

15.16 <br>
19.16 <br>
24.16 <br>
29.16 <br>

<BR>
Answer: 19.1672240937939

In [142]:
df['duration'] = df.dropOff_datetime - df.pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
df


,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,duration
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009,17.000000
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009,17.000000
2,B00013,2021-01-01 00:01:00,2021-01-01 01:51:00,NaN,NaN,None,B00013,110.000000
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,None,B00037,8.283333
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,None,B00037,15.216667
...,...,...,...,...,...,...,...,...
1154107,B03266,2021-01-31 23:43:03,2021-01-31 23:51:48,7.0,7.0,None,B03266,8.750000
1154108,B03284,2021-01-31 23:50:27,2021-02-01 00:48:03,44.0,91.0,None,,57.600000
1154109,B03285,2021-01-31 23:13:46,2021-01-31 23:29:58,171.0,171.0,None,B03285,16.200000
1154110,B03285,2021-01-31 23:58:03,2021-02-01 00:17:29,15.0,15.0,None,B03285,19.433333


In [143]:
df.duration.mean()

19.1672240937939

In [144]:
df.count()

dispatching_base_num      1154112
pickup_datetime           1154112
dropOff_datetime          1154112
PUlocationID               195845
DOlocationID               991892
SR_Flag                         0
Affiliated_base_number    1153227
duration                  1154112
dtype: int64

#### Data preparation
Check the distribution of the duration variable. There are some outliers.

Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

How many records did you drop?

In [145]:
df = df[(df.duration >= 1) & (df.duration <= 60)]
df

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,duration
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009,17.000000
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009,17.000000
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,None,B00037,8.283333
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,None,B00037,15.216667
5,B00037,2021-01-01 00:59:02,2021-01-01 01:08:05,NaN,71.0,None,B00037,9.050000
...,...,...,...,...,...,...,...,...
1154107,B03266,2021-01-31 23:43:03,2021-01-31 23:51:48,7.0,7.0,None,B03266,8.750000
1154108,B03284,2021-01-31 23:50:27,2021-02-01 00:48:03,44.0,91.0,None,,57.600000
1154109,B03285,2021-01-31 23:13:46,2021-01-31 23:29:58,171.0,171.0,None,B03285,16.200000
1154110,B03285,2021-01-31 23:58:03,2021-02-01 00:17:29,15.0,15.0,None,B03285,19.433333


In [146]:
df.count()

dispatching_base_num      1109826
pickup_datetime           1109826
dropOff_datetime          1109826
PUlocationID               182818
DOlocationID               961919
SR_Flag                         0
Affiliated_base_number    1109053
duration                  1109826
dtype: int64

In [147]:
1154112 - 1109826

44286

### There were 44286 outlier records dropped.

### Q3. Missing values
The features we'll use for our model are the pickup and dropoff location IDs.

But they have a lot of missing values there. Let's replace them with "-1".

What's the fractions of missing values for the pickup location ID? I.e. fraction of "-1"s after you filled the NAs.

53% <br>
63% <br>
73% <br>
83% <br>

Ans: 83% # 83.52%

In [164]:

categorical = ['PUlocationID', 'DOlocationID']
numerical = ['duration']

df[categorical]


,PUlocationID,DOlocationID
0,-1.0,-1.0
1,-1.0,-1.0
3,-1.0,72.0
4,-1.0,61.0
5,-1.0,71.0
...,...,...
1154107,7.0,7.0
1154108,44.0,91.0
1154109,171.0,171.0
1154110,15.0,15.0


In [87]:
df.count()

dispatching_base_num      1109826
pickup_datetime           1109826
dropOff_datetime          1109826
PUlocationID               182818
DOlocationID               961919
SR_Flag                         0
Affiliated_base_number    1109053
duration                  1109826
dtype: int64

In [149]:
df.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,duration
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009,17.000000
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009,17.000000
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,None,B00037,8.283333
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,None,B00037,15.216667
5,B00037,2021-01-01 00:59:02,2021-01-01 01:08:05,NaN,71.0,None,B00037,9.050000


In [150]:
df.dtypes

dispatching_base_num              object
pickup_datetime           datetime64[ns]
dropOff_datetime          datetime64[ns]
PUlocationID                     float64
DOlocationID                     float64
SR_Flag                           object
Affiliated_base_number            object
duration                         float64
dtype: object

In [157]:
df['PUlocationID'].isna().sum()

927008

In [155]:
# 927008 - Total num of NaN records
# 1109826 - Total num of records

(927008/1109826)*100

83.52732770722618

In [153]:
1109826 - 927008


182818

In [154]:
(182818/1109826)*100

16.472672292773822

In [165]:
df['PUlocationID'] = df['PUlocationID'].fillna(-1)
df['DOlocationID'] = df['DOlocationID'].fillna(-1)
df[categorical]


/tmp/ipykernel_18188/3400340561.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['PUlocationID'] = df['PUlocationID'].fillna(-1)
/tmp/ipykernel_18188/3400340561.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DOlocationID'] = df['DOlocationID'].fillna(-1)


,PUlocationID,DOlocationID
0,-1.0,-1.0
1,-1.0,-1.0
3,-1.0,72.0
4,-1.0,61.0
5,-1.0,71.0
...,...,...
1154107,7.0,7.0
1154108,44.0,91.0
1154109,171.0,171.0
1154110,15.0,15.0


### Q4. One-hot encoding
Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

* Turn the dataframe into a list of dictionaries
* Fit a dictionary vectorizer
* Get a feature matrix from it

What's the dimensionality of this matrix? (The number of columns).

2 <br>
152 <br>
352 <br>
525 <br>
725 <br>

Ans: 2 # Dimentions is (1109826, 2)

In [166]:
df[categorical + numerical].iloc[:10].to_dict(orient='records')

[{'PUlocationID': -1.0, 'DOlocationID': -1.0, 'duration': 17.0},
 {'PUlocationID': -1.0, 'DOlocationID': -1.0, 'duration': 17.0},
 {'PUlocationID': -1.0, 'DOlocationID': 72.0, 'duration': 8.283333333333333},
 {'PUlocationID': -1.0, 'DOlocationID': 61.0, 'duration': 15.216666666666667},
 {'PUlocationID': -1.0, 'DOlocationID': 71.0, 'duration': 9.05},
 {'PUlocationID': -1.0, 'DOlocationID': 91.0, 'duration': 11.866666666666667},
 {'PUlocationID': -1.0, 'DOlocationID': 39.0, 'duration': 8.883333333333333},
 {'PUlocationID': -1.0, 'DOlocationID': 37.0, 'duration': 17.966666666666665},
 {'PUlocationID': -1.0, 'DOlocationID': 39.0, 'duration': 23.366666666666667},
 {'PUlocationID': -1.0, 'DOlocationID': 72.0, 'duration': 2.033333333333333}]

In [167]:

categorical = ['PUlocationID', 'DOlocationID']
numerical = ['duration']
train_dicts = df[categorical].to_dict(orient='records')

dv = DictVectorizer()

X_train = dv.fit_transform(train_dicts)

target = 'duration'
y_train = df[target].values

In [168]:
train_dicts

[{'PUlocationID': -1.0, 'DOlocationID': -1.0},
 {'PUlocationID': -1.0, 'DOlocationID': -1.0},
 {'PUlocationID': -1.0, 'DOlocationID': 72.0},
 {'PUlocationID': -1.0, 'DOlocationID': 61.0},
 {'PUlocationID': -1.0, 'DOlocationID': 71.0},
 {'PUlocationID': -1.0, 'DOlocationID': 91.0},
 {'PUlocationID': -1.0, 'DOlocationID': 39.0},
 {'PUlocationID': -1.0, 'DOlocationID': 37.0},
 {'PUlocationID': -1.0, 'DOlocationID': 39.0},
 {'PUlocationID': -1.0, 'DOlocationID': 72.0},
 {'PUlocationID': -1.0, 'DOlocationID': 72.0},
 {'PUlocationID': -1.0, 'DOlocationID': 89.0},
 {'PUlocationID': -1.0, 'DOlocationID': 177.0},
 {'PUlocationID': -1.0, 'DOlocationID': 225.0},
 {'PUlocationID': -1.0, 'DOlocationID': 63.0},
 {'PUlocationID': -1.0, 'DOlocationID': 67.0},
 {'PUlocationID': -1.0, 'DOlocationID': 22.0},
 {'PUlocationID': -1.0, 'DOlocationID': 61.0},
 {'PUlocationID': -1.0, 'DOlocationID': 14.0},
 {'PUlocationID': -1.0, 'DOlocationID': 14.0},
 {'PUlocationID': -1.0, 'DOlocationID': 188.0},
 {'PUlocat

In [174]:
type(X_train[:1])

scipy.sparse.csr.csr_matrix

In [175]:
X_train.get_shape()

(1109826, 2)

### Q5. Training a model
Now let's use the feature matrix from the previous step to train a model.

Train a plain linear regression model with default parameters
Calculate the RMSE of the model on the training data

What's the RMSE on train?

5.52 <br>
10.52 <br>
15.52 <br>
20.52 <br>


In [177]:

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

11.415432830521663

### Q6. Evaluating the model
Now let's apply this model to the validation dataset (Feb 2021).

What's the RMSE on validation?

6.01 <br>
11.01 <br>
16.01 <br>
21.01 <br>


In [186]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.dropOff_datetime = pd.to_datetime(df.dropOff_datetime)
        df.pickup_datetime = pd.to_datetime(df.pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PUlocationID', 'DOlocationID']
    df[categorical] = df[categorical].astype(str)
    df['PUlocationID'] = df['PUlocationID'].fillna(-1)
    df['DOlocationID'] = df['DOlocationID'].fillna(-1)
    
    return df

In [187]:
df_train = read_dataframe('./data/fhv_tripdata_2021-01.parquet')
df_val = read_dataframe('./data/fhv_tripdata_2021-02.parquet')

In [188]:
len(df_train), len(df_val)

(1109826, 990113)

In [189]:
df_train['PU_DO'] = df_train['PUlocationID'] + '_' + df_train['DOlocationID']
df_val['PU_DO'] = df_val['PUlocationID'] + '_' + df_val['DOlocationID']

In [191]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['duration']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [192]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [193]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

4.321450208858706e-07

In [195]:
import pickle
with open('models/linear_reg_assign1.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [196]:
lr = Lasso(0.01)
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

0.0008972863324951596